In [3]:
push!(LOAD_PATH,pwd())
using CarMpcUtils
using Polynomials, Interpolations
using PyPlot
using PyCall
@pyimport matplotlib.patches as patches
pygui(true)

true

In [4]:
close("all")
zCurr = [-85.0, -108.0, 70.0*pi/180, 0.0]
Map = TrackMap("maps/RFS_2Lanes_Speed_0128.mat",nNodesPolyFront=25,nNodesPolyBack=10,nNodesThreshold=5)
vehicle = Vehicle(zCurr)
tuning = Tuning(dt = 0.2, dtMPC = 0.2, N = 20,
                Q = [0.5, 0.5, 10.0, 0.0], R = [20.0, 2.0],
                P = [1000.0, 20.0], vRef = 10.0, dSafe = 5.0,
                eYRef = 0.0, TTC = 3.0, eYSafe = 0.5)


CarMpcUtils.Tuning(0.2,0.2,20,20,[0.5,0.5,10.0,0.0],[20.0,2.0],[1000.0,20.0],10.0,[0.0],5.0,0.0,3.0,0.5)

In [ ]:
nodes, wayPointers, nLanes = Map.nodes, Map.wayPointers, Map.nWays
speed = Map.speed
nNodesPolyFront, nNodesPolyBack, nNodesThreshold = Map.nNodesPolyFront,
Map.nNodesPolyBack, Map.nNodesThreshold
xEgo, yEgo, psiEgo, vEgo = zCurr

# Polynomial fit for each lane
  coeffsLanes = Inf*ones(4,nLanes)
  idxClosestPointLanes = round(Int64,zeros(nLanes))
  for n=1:2
    # nodes for each lane
    nodesLane = nodes[:,wayPointers[1,n]:wayPointers[2,n]]
    nNodesLane = size(nodesLane,2)

    # distances from current position
    dists = sqrt(sum((nodesLane-repmat([xEgo,yEgo],1,nNodesLane)).^2,1))

    # distance and index of closest nodes
    minDist, minIdx = findmin(dists)
    idxClosestPointLanes[n] = wayPointers[1,n] + minIdx - 1
    minIdx = min(minIdx, nNodesLane-1)
    minIdx = max(minIdx, 2)

    # direction of motion
    dirEgo = [cos(psiEgo), sin(psiEgo)]
    dirPos = nodesLane[:,minIdx+1] - nodesLane[:,minIdx]
    direction = (dirEgo'*dirPos)[1] < 0 ? -1 : 1

    # nodes for polynomial fit
    idxStart = max(1, minIdx - (direction == 1 ? nNodesPolyBack : nNodesPolyFront))
    idxEnd = min(nNodesLane, minIdx + (direction == 1 ? nNodesPolyFront : nNodesPolyBack))
    nodesNear = nodesLane[:,idxStart:idxEnd]
    nNodesNear = size(nodesNear,2)

    if nNodesNear >= nNodesThreshold
      # transform points to ego frame
      R = [cos(psiEgo) -sin(psiEgo); sin(psiEgo) cos(psiEgo)];
      nodesLocal = R'*(nodesNear - repmat([xEgo,yEgo],1,nNodesNear))

      # compute least squares fit
      px = nodesLocal[1,:]'
      py = nodesLocal[2,:]'
      H = [ones(nNodesNear) px px.^2 px.^3]
      coeffsLanes[:,n] = -(H'*H)\H'*py
    end
  end

  # closest lane index
  minEy, laneIdx = findmin(abs(coeffsLanes[1,:]))
  coeffsCurrLane = coeffsLanes[:,laneIdx]
  mapSpeedRef = speed[idxClosestPointLanes[laneIdx]]

In [ ]:
ePsi = rad2deg(atan(coeffsCurrLane[2]))

In [ ]:
fig = figure()
plot(Map.nodes[1,:]',Map.nodes[2,:]')
# plot(nodesNear[1,:]',nodesNear[2,:]')
ax = gca()
a = 5.0
b = 2.0
# rect = patches.Rectangle((zCurr[1],zCurr[2]),20.0,10.0,angle=rad2deg(zCurr[3]))
# rect = patches.Ellipse((zCurr[1],zCurr[2]),a,b,angle=rad2deg(zCurr[3]))
rect = patches.Polygon(objectToVertices(vehicle)')
plot(zCurr[1],zCurr[2],marker="o")
plot(nodes[1,idxClosestPointLanes[laneIdx]],nodes[2,idxClosestPointLanes[laneIdx]],marker="s")
ax["add_patch"](rect)
show()

In [ ]:
figure()
plot(nodesLocal[1,:]',nodesLocal[2,:]')
grid("on")
xLocal = collect(-10.0:0.5:25.0)
yLocal = -polyval(Poly(coeffsCurrLane), xLocal)
plot(xLocal, yLocal)

In [ ]:
nodesNear

In [ ]:
nodesLocal

In [ ]:
laneIdx

In [ ]:
x = collect(0.0:0.1:10.0)

In [ ]:
diff(x)

In [ ]:
cumsum(diff(x))

In [ ]:
using CarMpc

In [ ]:
# parameters
  dt, N = tuning.dt, tuning.N
  xEgo, yEgo, psiEgo, vEgo = zCurr
  nz, nu = 4, 2

  # localize vehicle in lane and get reference speed
  coeffsCurrLane, laneIdx, vRef = localizeVehicle(zCurr,Map)
#   vRef = tuning.vRef

  # longitudinal distance along lane centerline as a function of X
  xLocal = collect(0.0:0.1:100.0)   # TODO: make these tuning parameters or part of map
  yLocal = polyval(Poly(-coeffsCurrLane),xLocal)
  sLocal = [0.0; cumsum(sqrt(diff(xLocal).^2 + diff(yLocal).^2))]

  # desired longitudinal positions
  sDesired = [0.0; cumsum(dt*vRef*ones(N))]

  # reference points on lane centerline
  itpX = interpolate((sLocal,), xLocal, Gridded(Linear()))
  itpY = interpolate((sLocal,), yLocal, Gridded(Linear()))
  xLaneLocal, yLaneLocal = itpX[sDesired], itpY[sDesired]
  posRefLocal = hcat(xLaneLocal,yLaneLocal)'

  # reference points in global frame
  R = [cos(psiEgo) -sin(psiEgo); sin(psiEgo) cos(psiEgo)]
  posRefGlobal = repmat([xEgo,yEgo],1,N+1) + R*posRefLocal

  # heading reference
  psiRef = psiEgo*ones(N+1,1) + [0.0; atan2(diff(xLaneLocal),diff(yLaneLocal))]

  # state reference
  ZRef = [posRefGlobal; psiRef'; vRef*ones(1,N+1)]
  URef = zeros(nu,N)

In [6]:
fig = figure()
plot(Map.nodes[1,:]',Map.nodes[2,:]')
# plot(ZRef[1,:]',ZRef[2,:]')

1-element Array{Any,1}:
 PyObject <matplotlib.lines.Line2D object at 0x31ec49d90>

In [ ]:
figure()
plot(xLocal,yLocal)
plot(xLaneLocal,yLaneLocal)

In [ ]:
figure()
plot(sLocal,xLocal)
plot(sDesired,xLaneLocal)

In [ ]:
fig = figure()
ax1 = fig["add_subplot"](111)
ax1["set_title"]("custom picker for line data", picker=true)
line = ax1["plot"](rand(100), rand(100), "o", picker=5)
@pyimport PlotUtils as plt_utils
fig["canvas"]["mpl_connect"]("pick_event", plt_utils.onpick1)
show()

In [9]:
pt1 = Map.nodes[:,Map.wayPointers[1,2]+50]
pt2 = Map.nodes[:,Map.wayPointers[1,2]+49]

2-element Array{Float64,1}:
 123.567 
  43.8112

In [10]:
pt1

2-element Array{Float64,1}:
 122.724 
  44.3005

In [13]:
atan2(pt1[2]-pt2[2],pt1[1]-pt2[1])

2.6152797189744503

In [15]:
z0 = vehicle.z

4-element Array{Float64,1}:
  -85.0    
 -108.0    
    1.22173
    0.0    

In [16]:
z0[1:2] = 0

0

In [17]:
z0

4-element Array{Float64,1}:
 0.0    
 0.0    
 1.22173
 0.0    

In [19]:
vehicle.z[1:2] = rand(2)

2-element Array{Float64,1}:
 0.627035
 0.678845

4-element Array{Float64,1}:
 0.627035
 0.678845
 1.22173 
 0.0     